# Create Json mask file to choose the appropriate threshold

Labelled exported as geoJSON:

* for QuPath coordinates need to be integers (pixels)
* coordinates need to be (x,y) and in form of a list (not nd-array)
* export of cells (around 10k cells) will take a while. Import to QuPath fast, as long as * no hierarchy is resolved

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os, sys

other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(other_dir_path)

print(other_dir_path)
import IMCprocess.spatial_analysis.spatial_analysis as sa
import IMCprocess.spatial_analysis.combi_enrich as ce
import IMCprocess.utils.utils as ut

# I have to use custom Img_anndata code
import Img_anndata as utan

/Users/pattama/Desktop/UiB/cancer/git_repo/IMCprocess


## Import mask and level1 annotated object

In [2]:
input_mask = 'sample_data/Patient1_001.tiff'
output_dir = 'output'
level1_file = 'sample_data/00hanna_adata_pilottma1_obj_Level1.pkl'
img_id = 'R1_001'

In [3]:
Img_obj_level1 = ut.read_pickle_obj(level1_file)
# Get adata level1 for img_id R1_001
Img_level1_R1_001 = ut.filter_adata_obs(Img_obj_level1.adata_dict['level1'],
                                        'img_id', 
                                        ['R1_001'])
# Get Epi adata
Epi_adata = ut.filter_adata_obs(Img_level1_R1_001,
                                'level1_annotated',
                                ['Ep'])

In [4]:
Epi_adata.obs

,img_id,PhenoGraph_clusters,level1_annotated
Cell_id,,,
14777_0,R1_001,16,Ep
14778_0,R1_001,1,Ep
14779_0,R1_001,1,Ep
14780_0,R1_001,1,Ep
14782_0,R1_001,1,Ep
...,...,...,...
18985_0,R1_001,1,Ep
18987_0,R1_001,7,Ep
18988_0,R1_001,6,Ep


In [6]:
# Check marker column name
Epi_adata.var_names

Index(['Bi209_pSTAT3_', 'Dy161_pSMAD2_', 'Dy162_CD8_', 'Dy163_ER_',
       'Dy164_CD20_', 'Er166_HER2_', 'Er167_GATA3_', 'Er168_Ki67_',
       'Er170_CD3_', 'Eu151_CD3134_', 'Eu153_CD44_', 'Gd155_FoxP3_',
       'Gd156_CD4_', 'Gd158_Ecad_', 'Gd160_PDGFRa_', 'In113_ADHL1_',
       'In115_GammaCat_', 'La139_MCAM_', 'Nd142_CTLA4_', 'Nd143_Vim_',
       'Nd144_YAP1_', 'Nd145_CD45_', 'Nd146_CD16_', 'Nd148_PanCk_',
       'Nd150_PDL1_', 'Pr141_aSMA_', 'Sm147_CD163_', 'Sm149_CK5_',
       'Sm152_FAP_', 'Sm154_CD11c_', 'Tb159_CD68_', 'Tm169_CD24_',
       'Yb171_PDGFRb_', 'Yb172_ClCasp3_', 'Yb173_GranzymeB_', 'Yb174_CK818_'],
      dtype='object')

## label 1, 0 for positive and negative cells from threshold

In [21]:
# User provide input marker column and 
# range of threshold for each marker
marker_threshold_dict = {'Er168_Ki67_': [0.2, 0.3, 0.4, 0.5],
                         'Nd143_Vim_': [0.2, 0.3, 0.4, 0.5],
                         'Tm169_CD24_': [0.2, 0.3, 0.4, 0.5]
}

In [ ]:
def binary_df(df, threshold):
    df_binary = df.applymap(lambda x: 1 if x >= threshold else 0)
    df_binary.columns = [col + '_'+ str(round(threshold,2)) for col in df.columns]
    return(df_binary)

def cell_neg_pos_dict(df, col):
    cell_neg_ind = np.array(df[df[col] == 0].index) + 1
    cell_pos_ind = np.array(df[df[col] == 1].index) + 1
    label_dict = {('neg_'+col): cell_neg_ind,
                  ('pos_'+col): cell_pos_ind}
    return(label_dict)

In [39]:
def create_json_marker(input_mask, input_adata, marker_col, 
                       threshold, output_dir):
    '''
    Create json file for specific marker and threshold
    '''
    # 1. get marker expression df
    marker_df = input_adata.to_df()[[marker_col]]
    # 2. get binary df
    bi_df = binary_df(marker_df, threshold).reset_index()
    # 3. get cell label dict
    cell_label_dict = cell_neg_pos_dict(bi_df, 
                                        marker_col + '_'+ str(round(threshold,2)))
    # 4. Check if output dir exist and crate mask
    if not os.path.exists(output_dir+ marker_col):
        os.makedirs(output_dir+ marker_col)
        print(f"{output_dir+marker_col} created successfully!")

    ut.export_labelmask_geoJson(input_mask, 
                                output_dir+ '/'+ marker_col + '/' + 
                                str(round(threshold,2)).replace('.','_') + '.json',
                                cell_label_dict,
                                [-1, -8245601]
                                )

In [40]:

create_json_marker(input_mask, 
                   Epi_adata, 
                   'Er168_Ki67_', 
                   0.2, output_dir+ '/Json_mask/')

output/Json_mask/Er168_Ki67_ created successfully!


In [41]:
marker_threshold_dict

{'Er168_Ki67_': [0.2, 0.3, 0.4, 0.5],
 'Nd143_Vim_': [0.2, 0.3, 0.4, 0.5],
 'Tm169_CD24_': [0.2, 0.3, 0.4, 0.5]}

In [43]:
for marker, threshold_list in marker_threshold_dict.items():
    print('create mask for ' + marker + 
          ' with threshold ' + str(threshold_list))
    for threshold in threshold_list:
        create_json_marker(input_mask, 
                           Epi_adata, 
                           marker, 
                           threshold, output_dir+ '/Json_mask/')

create mask for Er168_Ki67_ with threshold [0.2, 0.3, 0.4, 0.5]
create mask for Nd143_Vim_ with threshold [0.2, 0.3, 0.4, 0.5]
create mask for Tm169_CD24_ with threshold [0.2, 0.3, 0.4, 0.5]
